In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

# Сбор данных из SpaceX API V4


Функции для получения данных из API

In [2]:
# Берет набор данных и использует столбец rocket для вызова API и добавления данных в список
def getBoosterVersion(data):
    for x in data['rocket']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
            BoosterVersion.append(response['name'])

In [3]:
# Берет набор данных и использует столбец launchpad для вызова API и добавления данных в список
def getLaunchSite(data):
    for x in data['launchpad']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
            Longitude.append(response['longitude'])
            Latitude.append(response['latitude'])
            LaunchSite.append(response['name'])

In [4]:
# Берет набор данных и использует столбец полезных нагрузок для вызова API и добавления данных в списки
def getPayloadData(data):
    for load in data['payloads']:
        if load:
            response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
            PayloadMass.append(response['mass_kg'])
            Orbit.append(response['orbit'])

In [5]:
# Берет набор данных и использует столбец core для вызова API и добавления данных в списки
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Источник данных - [SpaceX REST API](https://github.com/r-spacex/SpaceX-API)

In [6]:
spacex_url="https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_url)

In [7]:
response.status_code

200

In [8]:
data = pd.json_normalize(response.json())
data.head()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,None,None,[],[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN


In [9]:
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]
data

,rocket,payloads,launchpad,cores,flight_number,date_utc
0,5e9d0d95eda69955f709d1eb,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",1,2006-03-24T22:30:00.000Z
1,5e9d0d95eda69955f709d1eb,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",2,2007-03-21T01:10:00.000Z
2,5e9d0d95eda69955f709d1eb,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",5e9e4502f5090995de566f86,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",3,2008-08-03T03:34:00.000Z
3,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",4,2008-09-28T23:15:00.000Z
4,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",5,2009-07-13T03:35:00.000Z
...,...,...,...,...,...,...
200,5e9d0d95eda69973a809d1ec,[],5e9e4501f509094ba4566f84,"[{'core': None, 'flight': None, 'gridfins': Tr...",198,2022-12-01T00:00:00.000Z
201,5e9d0d95eda69973a809d1ec,[63161760ffc78f3b8567071c],5e9e4502f509092b78566f87,"[{'core': None, 'flight': None, 'gridfins': No...",199,2022-12-01T00:00:00.000Z
202,5e9d0d95eda69973a809d1ec,[6243b896af52800c6e91926e],5e9e4501f509094ba4566f84,"[{'core': None, 'flight': None, 'gridfins': No...",200,2022-12-01T00:00:00.000Z
203,5e9d0d95eda69974db09d1ed,[],5e9e4502f509094188566f88,"[{'core': None, 'flight': None, 'gridfins': Tr...",202,2022-12-01T00:00:00.000Z


In [10]:
# Удалим строки с несколькими ядрами, потому что это ракеты falcon с двумя дополнительными ракетными ускорителями 
# и строки, которые содержат несколько полезных нагрузок в одной ракете.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]
data

,rocket,payloads,launchpad,cores,flight_number,date_utc
0,5e9d0d95eda69955f709d1eb,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",1,2006-03-24T22:30:00.000Z
1,5e9d0d95eda69955f709d1eb,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",2,2007-03-21T01:10:00.000Z
3,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",4,2008-09-28T23:15:00.000Z
4,5e9d0d95eda69955f709d1eb,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",5,2009-07-13T03:35:00.000Z
5,5e9d0d95eda69973a809d1ec,[5eb0e4b7b6c3bb0006eeb1e7],5e9e4501f509094ba4566f84,"[{'core': '5e9e289ef359185f2b3b2628', 'flight'...",6,2010-06-04T18:45:00.000Z
...,...,...,...,...,...,...
188,5e9d0d95eda69973a809d1ec,[631616aeffc78f3b8567071b],5e9e4501f509094ba4566f84,"[{'core': None, 'flight': None, 'gridfins': No...",188,2022-10-20T14:50:00.000Z
197,5e9d0d95eda69973a809d1ec,[6243b788af52800c6e91926b],5e9e4501f509094ba4566f84,"[{'core': None, 'flight': None, 'gridfins': No...",196,2022-11-01T00:00:00.000Z
201,5e9d0d95eda69973a809d1ec,[63161760ffc78f3b8567071c],5e9e4502f509092b78566f87,"[{'core': None, 'flight': None, 'gridfins': No...",199,2022-12-01T00:00:00.000Z
202,5e9d0d95eda69973a809d1ec,[6243b896af52800c6e91926e],5e9e4501f509094ba4566f84,"[{'core': None, 'flight': None, 'gridfins': No...",200,2022-12-01T00:00:00.000Z


In [11]:
# Полезная нагрузка и ядра представляют собой списки размера 1, извлекаем их
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])
data

,rocket,payloads,launchpad,cores,flight_number,date_utc
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight':...",1,2006-03-24T22:30:00.000Z
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,"{'core': '5e9e289ef35918416a3b2624', 'flight':...",2,2007-03-21T01:10:00.000Z
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,"{'core': '5e9e289ef3591855dc3b2626', 'flight':...",4,2008-09-28T23:15:00.000Z
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,"{'core': '5e9e289ef359184f103b2627', 'flight':...",5,2009-07-13T03:35:00.000Z
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,"{'core': '5e9e289ef359185f2b3b2628', 'flight':...",6,2010-06-04T18:45:00.000Z
...,...,...,...,...,...,...
188,5e9d0d95eda69973a809d1ec,631616aeffc78f3b8567071b,5e9e4501f509094ba4566f84,"{'core': None, 'flight': None, 'gridfins': Non...",188,2022-10-20T14:50:00.000Z
197,5e9d0d95eda69973a809d1ec,6243b788af52800c6e91926b,5e9e4501f509094ba4566f84,"{'core': None, 'flight': None, 'gridfins': Non...",196,2022-11-01T00:00:00.000Z
201,5e9d0d95eda69973a809d1ec,63161760ffc78f3b8567071c,5e9e4502f509092b78566f87,"{'core': None, 'flight': None, 'gridfins': Non...",199,2022-12-01T00:00:00.000Z
202,5e9d0d95eda69973a809d1ec,6243b896af52800c6e91926e,5e9e4501f509094ba4566f84,"{'core': None, 'flight': None, 'gridfins': Non...",200,2022-12-01T00:00:00.000Z


In [12]:
# Данные взятые из rocket, payload, launchpad, cores
# Записываются в глобальные переменные
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [13]:
getBoosterVersion(data)
getLaunchSite(data)
getPayloadData(data)
getCoreData(data)

In [14]:
# Создание Data Frame для последующего анализа
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': pd.to_datetime(data['date_utc']).dt.date,
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

df = pd.DataFrame(launch_dict)
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin2A,167.743129,9.047721
3,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin2C,167.743129,9.047721
4,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin3C,167.743129,9.047721
5,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0003,-80.577366,28.561857


In [15]:
df.BoosterVersion.value_counts()

Falcon 9    173
Falcon 1      4
Name: BoosterVersion, dtype: int64

In [16]:
# Удалим строки с Falcon 1
df = df[df.BoosterVersion != 'Falcon 1'].copy()
df.reset_index(inplace=True, drop=True)
df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0003,-80.577366,28.561857
1,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0005,-80.577366,28.561857
2,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0007,-80.577366,28.561857
3,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1.0,False,False,False,None,1.0,0.0,B1003,-120.610829,34.632093
4,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,188,2022-10-20,Falcon 9,13260.0,VLEO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857
169,196,2022-11-01,Falcon 9,5100.0,MEO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857
170,199,2022-12-01,Falcon 9,NaN,PO,VAFB SLC 4E,None None,NaN,None,None,None,None,NaN,NaN,None,-120.610829,34.632093
171,200,2022-12-01,Falcon 9,NaN,SSO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857


In [17]:
# Изменение нумерации запусков
df.loc[:,'FlightNumber'] = np.arange(1, df.shape[0]+1).astype('int64')
df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1.0,False,False,False,None,1.0,0.0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,169,2022-10-20,Falcon 9,13260.0,VLEO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857
169,170,2022-11-01,Falcon 9,5100.0,MEO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857
170,171,2022-12-01,Falcon 9,NaN,PO,VAFB SLC 4E,None None,NaN,None,None,None,None,NaN,NaN,None,-120.610829,34.632093
171,172,2022-12-01,Falcon 9,NaN,SSO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857


In [18]:
df.Date = pd.to_datetime(df.Date)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   FlightNumber    173 non-null    int64         
 1   Date            173 non-null    datetime64[ns]
 2   BoosterVersion  173 non-null    object        
 3   PayloadMass     149 non-null    float64       
 4   Orbit           172 non-null    object        
 5   LaunchSite      173 non-null    object        
 6   Outcome         173 non-null    object        
 7   Flights         168 non-null    float64       
 8   GridFins        168 non-null    object        
 9   Reused          168 non-null    object        
 10  Legs            168 non-null    object        
 11  LandingPad      142 non-null    object        
 12  Block           168 non-null    float64       
 13  ReusedCount     168 non-null    float64       
 14  Serial          168 non-null    object        
 15  Longit

# Сбор данных с сайта

Сайты источники  
[List of Falcon 9 and Falcon Heavy launches (2010–2019)](https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019))  
[List of Falcon 9 and Falcon Heavy launches](https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches)

In [20]:
# Получение всех интресующих таблиц
url = ['https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019)',
       'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches']
html = [requests.get(i).content for i in url]
soup = [BeautifulSoup(i, "html.parser") for i in html]

par = ('table', {'class': 'wikitable plainrowheaders collapsible'})
tables = soup[0].find_all(*par) + soup[1].find_all(*par)

In [21]:
# Функция извлекающая названия колонок таблицы
def extract_column_from_header(row):
    
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Отсортировываем пустыи и цифровые названия
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

In [22]:
first_launch_table = tables[0]
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

for col in first_launch_table.tr.find_all('th'):
    name = extract_column_from_header(col)
    if name and len(name) > 0:
        column_names.append(name)
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

In [23]:
launch_dict= dict.fromkeys(column_names)

# Удаляем ненужный ключ
del launch_dict['Date and time ( )']

# Инициализируем ключи пустыми списками
for i in launch_dict.keys():
    launch_dict[i] = []

# Добваляем несколько новых ключей с пустымы листами в качестве значений
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]

In [24]:
# Заполнение словаря launch_dict
for table in tables:
    for tr in table.find_all('tr'):
        if tr.th:
            if tr.th.string:
                flight_number = tr.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        if flag:

            row = tr.find_all('td')
            
            launch_dict['Flight No.'].append(int(flight_number))
            
            temp_l_s = row[2].text.strip().replace(',', ', ')
            launch_site = re.sub(r'\[\w*\]', '', temp_l_s)
            launch_dict['Launch site'].append(launch_site)
            
            payload = row[3].a.text
            launch_dict['Payload'].append(payload)
            
            payload_mass = row[4]
            index_number = payload_mass.text.find('kg')
            if index_number != -1:
                payload_mass = payload_mass.text[0: index_number + 2].strip(' ~kg').replace(',', '').replace(u'\xa0','')
            else:
                payload_mass = list(payload_mass.strings)[0].strip()
            launch_dict['Payload mass'].append(payload_mass)
            
            orbit = re.sub(r'\[\w*\]', '', row[5].text).strip()
            launch_dict['Orbit'].append(orbit)
            
            temp_customer = ' '.join([k.strip() for k in row[6].strings 
                                     if (k.find('[') == -1) and (k != '\n')]).strip(' _\n').replace('\n', ' ')
            customer = temp_customer
            launch_dict['Customer'].append(customer)
            
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)
            
            version_booster = '-'.join([k for k in row[1].strings 
                                        if (k.find('[') == -1) and (k.find('♺') == -1) 
                                        and (k != '\n') and (k != ' ')]).strip()
            launch_dict['Version Booster'].append(version_booster)
            
            temp_b_l = re.sub(r'\[\w*\]', '', row[8].text).strip()
            booster_landing = temp_b_l if re.match(r'[A-z ]* \(', temp_b_l) else re.sub(r'\(', ' (', temp_b_l)
            launch_dict['Booster landing'].append(booster_landing)
            
            if row[0].a: row[0].a.extract()
            if row[0].a: row[0].a.extract()
            date_time = list(row[0].strings)[0:2]
            
            date = date_time[0].strip(',')
            launch_dict['Date'].append(date)

            time = date_time[1].strip()

# Преобразование launch_dict в Data Frame
df_web = pd.DataFrame(launch_dict)
df_web.Date = pd.to_datetime(df_web.Date)
df_web

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date
0,1,"CCAFS, SLC-40",Dragon Spacecraft Qualification Unit,No payload (excl. Dragon Mass),LEO,SpaceX,Success,F9 v1.0-B0003,Failure (parachute),2010-06-04
1,2,"CCAFS, SLC-40",SpaceX COTS Demo Flight 1,Unknown (excl. Dragon Mass),LEO (ISS),NASA ( COTS ) various others,Success,F9 v1.0-B0004,Failure (parachute),2010-12-08
2,3,"CCAFS, SLC-40",SpaceX COTS Demo Flight 2,525,LEO (ISS),NASA ( COTS ),Success,F9 v1.0-B0005,No attempt,2012-05-22
3,4,"CCAFS, SLC-40",SpaceX CRS-1,4700,LEO (ISS),NASA ( CRS ),Success,F9 v1.0-B0006,No attempt,2012-10-08
4,5,"CCAFS, SLC-40",SpaceX CRS-2,4877,LEO (ISS),NASA ( CRS ),Success,F9 v1.0-B0007,No attempt,2013-03-01
...,...,...,...,...,...,...,...,...,...,...
190,191,"CCSFS, SLC-40",O3b mPOWER,4100,MEO,SES,Success,F9 B5-B1067.8,Success (drone ship),2022-12-16
191,192,"KSC, LC-39A",Starlink Group 4-37,16700,LEO,SpaceX,Success,F9 B5-B1058.15,Success (drone ship),2022-12-17
192,193,"CCSFS, SLC-40",Starlink Group 5-1,16700,LEO,SpaceX,Success,F9 B5-B1062.11,Success (drone ship),2022-12-28
193,194,"VSFB, SLC-4E",EROS-C3,400,Retrograde LEO,ImageSat International,Success,F9 B5-B1061.11,Success (ground pad),2022-12-30


In [25]:
pd.unique(df_web['Payload mass'])

array(['No payload (excl. Dragon Mass)', 'Unknown (excl. Dragon Mass)',
       '525', '4700', '4877', '500', '3170', '3325', '2296', '1316',
       '4535', '4428', '2216', '2395', '570', '4159', '1898', '4707',
       '1952', '2034', '553', '5271', '3136', '4696', '3100', '3600',
       '2257', '4600', '9600', '2490', '5600', '5300', 'Classified',
       '6070', '2708', '3669', '6761', '3310', '475', '4990', '5400',
       '3500', '2205', '4230', '2150', '6092', '2647', '362', '6460',
       '5384', '2697', '7075', '5800', '7060', '3000', '4000', '2500',
       '4400', '4850', '12055', '2495', '13620', '4200', '2268', '6500',
       '15600', '2617', '6956', '12050', '1977', '12530', '15410', '4311',
       '5000–6000', '14932', '15440', '3130', '12500', '1192', '2972',
       '7000', '5000', '13000', '14000', '3328', '4331', 'Unknown',
       '2200', '13260', '12519', '15635', '624', '14500', '325', '4500',
       '2989', '13600', '14750', '13900', '14160', '16250', '4100', '700',
    

In [26]:
# Функция для извлечения чисел
def str_to_int(x):
    reg = re.search('5000–6000', x)
    if x.isdigit() or reg:
        return int(x) if not reg else 5500
    else:
        return None
    
df_web["Payload mass"] = df_web["Payload mass"].apply(str_to_int)

In [27]:
pd.unique(df_web["Payload mass"])

array([   nan,   525.,  4700.,  4877.,   500.,  3170.,  3325.,  2296.,
        1316.,  4535.,  4428.,  2216.,  2395.,   570.,  4159.,  1898.,
        4707.,  1952.,  2034.,   553.,  5271.,  3136.,  4696.,  3100.,
        3600.,  2257.,  4600.,  9600.,  2490.,  5600.,  5300.,  6070.,
        2708.,  3669.,  6761.,  3310.,   475.,  4990.,  5400.,  3500.,
        2205.,  4230.,  2150.,  6092.,  2647.,   362.,  6460.,  5384.,
        2697.,  7075.,  5800.,  7060.,  3000.,  4000.,  2500.,  4400.,
        4850., 12055.,  2495., 13620.,  4200.,  2268.,  6500., 15600.,
        2617.,  6956., 12050.,  1977., 12530., 15410.,  4311.,  5500.,
       14932., 15440.,  3130., 12500.,  1192.,  2972.,  7000.,  5000.,
       13000., 14000.,  3328.,  4331.,  2200., 13260., 12519., 15635.,
         624., 14500.,   325.,  4500.,  2989., 13600., 14750., 13900.,
       14160., 16250.,  4100.,   700., 14100.,  2668.,   679., 16000.,
       16700., 14200., 11938., 16100.,  7350.,  4501., 16400.,  6600.,
      

# Предобоработка данных

In [28]:
df.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass       24
Orbit              1
LaunchSite         0
Outcome            0
Flights            5
GridFins           5
Reused             5
Legs               5
LandingPad        31
Block              5
ReusedCount        5
Serial             5
Longitude          0
Latitude           0
dtype: int64

In [29]:
df.tail(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
163,164,2022-08-28,Falcon 9,13260.0,VLEO,KSC LC 39A,True ASDS,2.0,True,True,True,5e9e3033383ecb075134e7cd,5.0,1.0,B1069,-80.603956,28.608058
164,165,2022-08-31,Falcon 9,13260.0,VLEO,VAFB SLC 4E,True ASDS,7.0,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,6.0,B1063,-120.610829,34.632093
165,166,2022-09-17,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,6.0,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,5.0,B1067,-80.577366,28.561857
166,167,2022-09-24,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,4.0,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,0.0,B1072,-80.577366,28.561857
167,168,2022-10-05,Falcon 9,NaN,ISS,KSC LC 39A,True ASDS,1.0,True,False,True,5e9e3033383ecbb9e534e7cc,5.0,0.0,B1077,-80.603956,28.608058
168,169,2022-10-20,Falcon 9,13260.0,VLEO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857
169,170,2022-11-01,Falcon 9,5100.0,MEO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857
170,171,2022-12-01,Falcon 9,NaN,PO,VAFB SLC 4E,None None,NaN,None,None,None,None,NaN,NaN,None,-120.610829,34.632093
171,172,2022-12-01,Falcon 9,NaN,SSO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857
172,173,2022-12-01,Falcon 9,5100.0,MEO,CCSFS SLC 40,None None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857


In [30]:
# Удаление 5 нижних строк, из-за невозможности заполнить пропуски
df = df.iloc[:-5, :].copy()

df.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass       22
Orbit              1
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [31]:
temp_union = df.merge(df_web, how='left', on='Date', suffixes = ['API', 'WEB'])
temp_union

,FlightNumber,Date,BoosterVersion,PayloadMass,OrbitAPI,LaunchSite,Outcome,Flights,GridFins,Reused,...,Latitude,Flight No.,Launch site,Payload,Payload mass,OrbitWEB,Customer,Launch outcome,Version Booster,Booster landing
0,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1.0,False,False,...,28.561857,1.0,"CCAFS, SLC-40",Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,F9 v1.0-B0003,Failure (parachute)
1,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1.0,False,False,...,28.561857,3.0,"CCAFS, SLC-40",SpaceX COTS Demo Flight 2,525.0,LEO (ISS),NASA ( COTS ),Success,F9 v1.0-B0005,No attempt
2,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1.0,False,False,...,28.561857,5.0,"CCAFS, SLC-40",SpaceX CRS-2,4877.0,LEO (ISS),NASA ( CRS ),Success,F9 v1.0-B0007,No attempt
3,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1.0,False,False,...,34.632093,6.0,"VAFB, SLC-4E",CASSIOPE,500.0,Polar orbit LEO,MDA,Success,F9 v1.1-B1003,Uncontrolled (ocean)
4,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1.0,False,False,...,28.561857,7.0,"CCAFS, SLC-40",SES-8,3170.0,GTO,SES,Success,F9 v1.1-B1004,No attempt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,165,2022-08-31,Falcon 9,13260.0,VLEO,VAFB SLC 4E,True ASDS,7.0,True,True,...,34.632093,173.0,"VSFB, SLC-4E",Starlink Group 3-4,14200.0,SSO,SpaceX,Success,F9 B5-B1063.7,Success (drone ship)
165,166,2022-09-17,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,6.0,True,True,...,28.561857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,167,2022-09-24,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,4.0,True,True,...,28.561857,177.0,"CCSFS, SLC-40",Starlink Group 4-35,16100.0,LEO,SpaceX,Success,F9 B5-B1073.4,Success (drone ship)
167,168,2022-10-05,Falcon 9,NaN,ISS,KSC LC 39A,True ASDS,1.0,True,False,...,28.608058,178.0,"KSC, LC-39A",Crew-5,13000.0,LEO (ISS),NASA ( CTS ),Success,F9 B5-B1077.1,Success (drone ship)


In [32]:
temp_union.columns

Index(['FlightNumber', 'Date', 'BoosterVersion', 'PayloadMass', 'OrbitAPI',
       'LaunchSite', 'Outcome', 'Flights', 'GridFins', 'Reused', 'Legs',
       'LandingPad', 'Block', 'ReusedCount', 'Serial', 'Longitude', 'Latitude',
       'Flight No.', 'Launch site', 'Payload', 'Payload mass', 'OrbitWEB',
       'Customer', 'Launch outcome', 'Version Booster', 'Booster landing'],
      dtype='object')

In [33]:
df_web = temp_union[['Flight No.', 'Date', 'Launch site', 'Payload', 'Payload mass', 'OrbitWEB',
                     'Customer', 'Launch outcome', 'Version Booster', 'Booster landing']].iloc[:-1,:].copy()
df_web.rename(columns={'OrbitWEB': 'Orbit'}, inplace=True)

In [34]:
temp_union = temp_union[['FlightNumber', 'Flight No.', 'BoosterVersion', 'Version Booster', 'PayloadMass', 'Payload mass',
                         'OrbitAPI', 'OrbitWEB', 'LaunchSite', 'Launch site', 'Outcome', 'Launch outcome']]
temp_union

,FlightNumber,Flight No.,BoosterVersion,Version Booster,PayloadMass,Payload mass,OrbitAPI,OrbitWEB,LaunchSite,Launch site,Outcome,Launch outcome
0,1,1.0,Falcon 9,F9 v1.0-B0003,NaN,NaN,LEO,LEO,CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
1,2,3.0,Falcon 9,F9 v1.0-B0005,525.0,525.0,LEO,LEO (ISS),CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
2,3,5.0,Falcon 9,F9 v1.0-B0007,677.0,4877.0,ISS,LEO (ISS),CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
3,4,6.0,Falcon 9,F9 v1.1-B1003,500.0,500.0,PO,Polar orbit LEO,VAFB SLC 4E,"VAFB, SLC-4E",False Ocean,Success
4,5,7.0,Falcon 9,F9 v1.1-B1004,3170.0,3170.0,GTO,GTO,CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
...,...,...,...,...,...,...,...,...,...,...,...,...
164,165,173.0,Falcon 9,F9 B5-B1063.7,13260.0,14200.0,VLEO,SSO,VAFB SLC 4E,"VSFB, SLC-4E",True ASDS,Success
165,166,NaN,Falcon 9,NaN,13260.0,NaN,VLEO,NaN,CCSFS SLC 40,NaN,True ASDS,NaN
166,167,177.0,Falcon 9,F9 B5-B1073.4,13260.0,16100.0,VLEO,LEO,CCSFS SLC 40,"CCSFS, SLC-40",True ASDS,Success
167,168,178.0,Falcon 9,F9 B5-B1077.1,NaN,13000.0,ISS,LEO (ISS),KSC LC 39A,"KSC, LC-39A",True ASDS,Success


In [35]:
# Удаление последней строчки
temp_union = temp_union.iloc[:-1, :]
temp_union.FlightNumber.value_counts()

1      1
116    1
108    1
109    1
110    1
      ..
58     1
59     1
60     1
61     1
168    1
Name: FlightNumber, Length: 168, dtype: int64

In [36]:
temp_union

,FlightNumber,Flight No.,BoosterVersion,Version Booster,PayloadMass,Payload mass,OrbitAPI,OrbitWEB,LaunchSite,Launch site,Outcome,Launch outcome
0,1,1.0,Falcon 9,F9 v1.0-B0003,NaN,NaN,LEO,LEO,CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
1,2,3.0,Falcon 9,F9 v1.0-B0005,525.0,525.0,LEO,LEO (ISS),CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
2,3,5.0,Falcon 9,F9 v1.0-B0007,677.0,4877.0,ISS,LEO (ISS),CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
3,4,6.0,Falcon 9,F9 v1.1-B1003,500.0,500.0,PO,Polar orbit LEO,VAFB SLC 4E,"VAFB, SLC-4E",False Ocean,Success
4,5,7.0,Falcon 9,F9 v1.1-B1004,3170.0,3170.0,GTO,GTO,CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
...,...,...,...,...,...,...,...,...,...,...,...,...
163,164,172.0,Falcon 9,F9 B5-B1069.2,13260.0,16700.0,VLEO,LEO,KSC LC 39A,"CCSFS, SLC-40",True ASDS,Success
164,165,173.0,Falcon 9,F9 B5-B1063.7,13260.0,14200.0,VLEO,SSO,VAFB SLC 4E,"VSFB, SLC-4E",True ASDS,Success
165,166,NaN,Falcon 9,NaN,13260.0,NaN,VLEO,NaN,CCSFS SLC 40,NaN,True ASDS,NaN
166,167,177.0,Falcon 9,F9 B5-B1073.4,13260.0,16100.0,VLEO,LEO,CCSFS SLC 40,"CCSFS, SLC-40",True ASDS,Success


In [37]:
df.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass       22
Orbit              1
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [38]:
temp_union = temp_union.loc[df.PayloadMass.isnull() | df.Orbit.isnull()]
temp_union

,FlightNumber,Flight No.,BoosterVersion,Version Booster,PayloadMass,Payload mass,OrbitAPI,OrbitWEB,LaunchSite,Launch site,Outcome,Launch outcome
0,1,1.0,Falcon 9,F9 v1.0-B0003,NaN,NaN,LEO,LEO,CCSFS SLC 40,"CCAFS, SLC-40",None None,Success
29,30,33.0,Falcon 9,F9 FT-B1032.1,NaN,NaN,LEO,LEO,KSC LC 39A,"KSC, LC-39A",True RTLS,Success
43,44,47.0,Falcon 9,F9 B4-B1043.1,NaN,NaN,LEO,LEO,CCSFS SLC 40,"CCAFS, SLC-40",True RTLS,Success
72,73,79.0,Falcon 9,F9 B5-B1046.4,NaN,12050.0,SO,Sub-orbital,KSC LC 39A,"KSC, LC-39A",None None,Success
82,83,89.0,Falcon 9,F9 B5-B1058.2,NaN,5500.0,GEO,GTO,CCSFS SLC 40,"CCAFS, SLC-40",True ASDS,Success
90,91,98.0,Falcon 9,F9 B5-B1061.1,NaN,12500.0,ISS,LEO (ISS),KSC LC 39A,"KSC, LC-39A",True ASDS,Success
95,96,103.0,Falcon 9,F9 B5-B1059.5,NaN,NaN,None,LEO,KSC LC 39A,"KSC, LC-39A",True RTLS,Success
98,99,106.0,Falcon 9,F9 B5-B1058.5,NaN,5000.0,SSO,SSO,CCSFS SLC 40,"CCSFS, SLC-40",True ASDS,Success
106,107,114.0,Falcon 9,F9 B5-B1061.2,NaN,13000.0,ISS,LEO (ISS),KSC LC 39A,"KSC, LC-39A",True ASDS,Success
114,115,123.0,Falcon 9,F9 B5-B1060.8,NaN,NaN,SSO,SSO,CCSFS SLC 40,"CCSFS, SLC-40",True RTLS,Success


In [39]:
# Заполнение пробелов в df данными из df_web
for i, serias in temp_union.iterrows():
    df.loc[i, 'PayloadMass'] = serias['Payload mass'] if np.isnan(serias['PayloadMass']) else serias['PayloadMass']
    df.loc[i, 'Orbit'] = serias['OrbitWEB'] if not (serias['OrbitAPI']) else serias['OrbitAPI']

In [40]:
df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1.0,False,False,False,None,1.0,0.0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,164,2022-08-28,Falcon 9,13260.0,VLEO,KSC LC 39A,True ASDS,2.0,True,True,True,5e9e3033383ecb075134e7cd,5.0,1.0,B1069,-80.603956,28.608058
164,165,2022-08-31,Falcon 9,13260.0,VLEO,VAFB SLC 4E,True ASDS,7.0,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,6.0,B1063,-120.610829,34.632093
165,166,2022-09-17,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,6.0,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,5.0,B1067,-80.577366,28.561857
166,167,2022-09-24,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,4.0,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,0.0,B1072,-80.577366,28.561857


In [41]:
df.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass       10
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [42]:
# Оставшиеся пропуски полезной нагрузки заменяем средним арифметическим значением
PM_mean = df.PayloadMass.mean()

df.loc[:, 'PayloadMass'].replace(np.nan, PM_mean, inplace=True)
df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,8248.883228,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1.0,False,False,False,None,1.0,0.0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,164,2022-08-28,Falcon 9,13260.000000,VLEO,KSC LC 39A,True ASDS,2.0,True,True,True,5e9e3033383ecb075134e7cd,5.0,1.0,B1069,-80.603956,28.608058
164,165,2022-08-31,Falcon 9,13260.000000,VLEO,VAFB SLC 4E,True ASDS,7.0,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,6.0,B1063,-120.610829,34.632093
165,166,2022-09-17,Falcon 9,13260.000000,VLEO,CCSFS SLC 40,True ASDS,6.0,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,5.0,B1067,-80.577366,28.561857
166,167,2022-09-24,Falcon 9,13260.000000,VLEO,CCSFS SLC 40,True ASDS,4.0,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,0.0,B1072,-80.577366,28.561857


In [43]:
df.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [44]:
df_web["Flight No."] = df["FlightNumber"]
df_web["Launch site"] = df['LaunchSite']
df_web["Payload mass"] = df['PayloadMass']
df_web['Orbit'] = df['Orbit']

In [45]:
# Процент пропущенных значений
df.isnull().sum()/df.count()*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        18.309859
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

In [46]:
df.dtypes

FlightNumber               int64
Date              datetime64[ns]
BoosterVersion            object
PayloadMass              float64
Orbit                     object
LaunchSite                object
Outcome                   object
Flights                  float64
GridFins                  object
Reused                    object
Legs                      object
LandingPad                object
Block                    float64
ReusedCount              float64
Serial                    object
Longitude                float64
Latitude                 float64
dtype: object

In [47]:
df.to_pickle("./data/df.pkl")

In [48]:
df_web.to_pickle("./data/df_web.pkl")